In [1]:
import pandas as pd
import numpy as np


df = pd.read_excel('Data_Train.xlsx')

df.head()


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [2]:
df.columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info', 'Price'],
      dtype='object')

### ✈️ Dataset Columns Overview

| Column           | Description                                   | Usefulness for Anomaly Detection                |
|------------------|-----------------------------------------------|-------------------------------------------------|
| `Airline`        | Airline name (e.g., Jet Airways)              | Grouping factor for price norms                 |
| `Date_of_Journey`| Travel date                                   | Needed for **seasonality & trends**             |
| `Source`         | Origin city                                   | Helps detect location-based patterns            |
| `Destination`    | Arrival city                                  | Same as above                                   |
| `Route`          | Route taken (e.g., DEL → BOM → BLR)           | Can indicate complexity of trip                 |
| `Dep_Time`       | Departure time                                | Helps track **time-of-day pricing**             |
| `Arrival_Time`   | Arrival time                                  | Less useful, but can help with duration         |
| `Duration`       | Flight duration                               | Longer flights usually cost more                |
| `Total_Stops`    | Number of stops (e.g., non-stop, 1 stop)      | Strong price factor                             |
| `Additional_Info`| Misc (e.g., "No info", "In-flight meal not included") | Might explain anomalies                |
| `Price`          | 💰**Target variable**                         | We’ll analyze this for **outliers**             |


In [3]:
# Convert 'Date' column to datetime format
df['Date_of_Journey'] = pd.to_datetime(df['Date_of_Journey'], format='%d/%m/%Y')

# Extract year, month, and day from the 'Date' column
df['Journey_day'] = df['Date_of_Journey'].dt.day
df['Journey_month'] = df['Date_of_Journey'].dt.month
df['Journey_weekday'] = df['Date_of_Journey'].dt.weekday

# Convert 'Deep_Time' to hours and minutes
df['Dep_hour'] = pd.to_datetime(df['Dep_Time']).dt.hour
df['Dep_minute'] = pd.to_datetime(df['Dep_Time']).dt.minute

# Clean Duration: Convert to total minutes
# Some values only have hours or minutes, so we need to handle those cases
df['Duration'] = df['Duration'].apply(lambda x: x.replace('h', 'h ')).replace('m', 'm ') # normalize spacing

def duration_to_minutes(duration):
    hours = 0
    minutes = 0
    for part in duration.strip().split():
        if 'h' in part:
            hours = int(part.replace('h', ''))
        elif 'm' in part:
            minutes = int(part.replace('m', ''))
    return hours * 60 + minutes

df['Duration_mins'] = df['Duration'].apply(duration_to_minutes)

C:\Users\DELL XPS\AppData\Local\Temp\ipykernel_4888\2436003973.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dep_hour'] = pd.to_datetime(df['Dep_Time']).dt.hour
C:\Users\DELL XPS\AppData\Local\Temp\ipykernel_4888\2436003973.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dep_minute'] = pd.to_datetime(df['Dep_Time']).dt.minute


In [4]:
df[['Date_of_Journey', 'Journey_day', 'Journey_month', 'Journey_weekday', 'Dep_Time', 'Dep_hour', 'Duration', 'Duration_mins']].head()


,Date_of_Journey,Journey_day,Journey_month,Journey_weekday,Dep_Time,Dep_hour,Duration,Duration_mins
0,2019-03-24,24,3,6,22:20,22,2h 50m,170
1,2019-05-01,1,5,2,05:50,5,7h 25m,445
2,2019-06-09,9,6,6,09:25,9,19h,1140
3,2019-05-12,12,5,6,18:05,18,5h 25m,325
4,2019-03-01,1,3,4,16:50,16,4h 45m,285


#### IQR Method for Skwewed Data

In [6]:
# IQR method
Q1 = df['Price'].quantile(0.25)
Q3 = df['Price'].quantile(0.75)
IQR = Q3 - Q1

# Define threshold
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Flag anomalies
df['Price_Anomaly_IQR'] = df['Price'].apply(lambda x: 'Anomaly' if x < lower_bound or x > upper_bound else 'Normal')


# See how many anomalies we have
anomaly_count = df['Price_Anomaly_IQR'].value_counts()

anomaly_count

Price_Anomaly_IQR
Normal     10589
Anomaly       94
Name: count, dtype: int64

#### Z-Score Method


In [8]:
from scipy.stats import zscore

# Compute z-scores
df['Price_zscore'] = zscore(df['Price'])

# Flag extreme outliers (typical z > 3 or < -3)
df['Price_Anomaly_Z'] = df['Price_zscore'].apply(lambda x: 'Anomaly' if abs(x) > 3 else 'Normal')

# See results
df['Price_Anomaly_Z'].value_counts()

Price_Anomaly_Z
Normal     10587
Anomaly       96
Name: count, dtype: int64

In [9]:
# Using IQR
df[df['Price_Anomaly_IQR'] == 'Anomaly'][['Airline', 'Source', 'Destination', 'Duration', 'Total_Stops', 'Price']].sort_values(by='Price')

# Or using Z-score
df[df['Price_Anomaly_Z'] == 'Anomaly'][['Airline', 'Source', 'Destination', 'Duration', 'Total_Stops', 'Price']].sort_values(by='Price')


,Airline,Source,Destination,Duration,Total_Stops,Price
4676,Jet Airways,Mumbai,Hyderabad,23h 35m,2 stops,22950
8856,Jet Airways,Banglore,New Delhi,7h,2 stops,23001
8020,Multiple carriers,Delhi,Cochin,15h 10m,1 stop,23170
5136,SpiceJet,Banglore,New Delhi,7h 55m,1 stop,23267
8990,Jet Airways,Mumbai,Hyderabad,10h 5m,2 stops,23528
...,...,...,...,...,...,...
1478,Jet Airways,Banglore,New Delhi,6h 5m,1 stop,54826
2618,Jet Airways,Banglore,New Delhi,6h 15m,1 stop,54826
10364,Jet Airways Business,Banglore,New Delhi,4h 40m,1 stop,57209
5372,Jet Airways Business,Banglore,New Delhi,6h 40m,1 stop,62427


#### This tells us:

- Both methods agree pretty well (only a small difference).

- Roughly 0.9% of the flights were flagged as pricing outliers.

- We're now able to detect price anomalies in a clean and explainable way — no black-box needed 🔥



### High-Opportunity Flights (Scoring System)

Let’s say your goal is to **flag the best “deals”** (e.g., prices that are **unusually low** given the route, duration, etc.).

We can build a simple **price efficiency** score.


#### 💡 Idea: Price per Minute Score (lower = better deal)


In [10]:
# Avoid division by zero
df['Price_per_minute'] = df['Price'] / df['Duration_mins'].replace(0, np.nan)

# Sort by best deala
best_deals = df.sort_values(by='Price_per_minute').head()

# Show top "cheap per minute" flights
best_deals[['Airline', 'Source', 'Destination', 'Duration', 'Price', 'Price_per_minute']]

,Airline,Source,Destination,Duration,Price,Price_per_minute
10566,Air India,Delhi,Cochin,27h 20m,4487,2.735976
274,Air India,Delhi,Cochin,27h 20m,4487,2.735976
1571,Air India,Kolkata,Banglore,25h 55m,4647,2.988424
61,Air India,Delhi,Cochin,27h 20m,5117,3.120122
9819,Air India,Delhi,Cochin,27h 20m,5117,3.120122


In [11]:
# Cheap and unusual
df[(df['Price_Anomaly_IQR'] == 'Anomaly') & (df['Price_per_minute'] < df['Price_per_minute'].quantile(0.25))][
    ['Airline', 'Source', 'Destination', 'Duration', 'Price', 'Price_per_minute']
].sort_values(by='Price_per_minute')


,Airline,Source,Destination,Duration,Price,Price_per_minute


### 🔍 What Could Be Happening?
1. Most "anomalies" were likely high prices, not low deals.

2. Our lowest Price_per_minute flights might be within the normal price range — not flagged as anomalies.

Let’s check this step-by-step to confirm:


### Step 1: Were Most IQR Anomalies High Prices

In [12]:
# Check the prices of IQR anomalies
df[df['Price_Anomaly_IQR'] == 'Anomaly']['Price'].describe()

count       94.000000
mean     30423.042553
std       9652.314368
min      23170.000000
25%      25735.000000
50%      26890.000000
75%      31219.250000
max      79512.000000
Name: Price, dtype: float64

✅ All 94 price anomalies flagged by the IQR method are high-priced flights — not underpriced deals.

Let’s break this down:

| Metric        | Value  |
| ------------- | ------ |
| **Min Price** | 23,170 |
| **Max Price** | 79,512 |
| **Median**    | 26,890 |
| **Mean**      | 30,423 |
| **Std Dev**   | 9,652  |

✅ This tells us:

- All price anomalies are on the higher end.

- Even the lowest “anomaly” is $23,170, which is above the overall IQR threshold.

**✅ What to Do Instead**
Since there are no cheap flights being flagged as anomalies, let’s focus on finding great deals even if they’re not anomalies.

#### 💡 Show the Cheapest Flights (Price per Minute)

In [13]:
df.sort_values(by='Price_per_minute').head(10)[
    ['Airline', 'Source', 'Destination', 'Duration', 'Price', 'Price_per_minute']
]


,Airline,Source,Destination,Duration,Price,Price_per_minute
10566,Air India,Delhi,Cochin,27h 20m,4487,2.735976
274,Air India,Delhi,Cochin,27h 20m,4487,2.735976
1571,Air India,Kolkata,Banglore,25h 55m,4647,2.988424
61,Air India,Delhi,Cochin,27h 20m,5117,3.120122
9819,Air India,Delhi,Cochin,27h 20m,5117,3.120122
926,Air India,Delhi,Cochin,26h 30m,5022,3.158491
6747,Air India,Delhi,Cochin,26h 30m,5022,3.158491
4591,Jet Airways,Delhi,Cochin,27h 35m,5406,3.266465
5230,Jet Airways,Delhi,Cochin,26h 10m,5177,3.297452
5346,Jet Airways,Delhi,Cochin,33h 15m,6643,3.329825


### 🔁 Optional: Flag “Good Deals” Based on Logic

In [14]:
# Define a custom threshold for "good deals"
threshold = df['Price_per_minute'].quantile(0.10)

df['Deal_Flag'] = df['Price_per_minute'].apply(lambda x: 'Good Deal' if x < threshold else 'Regular')

# View some flagged deals
df[df['Deal_Flag'] == 'Good Deal'][
    ['Airline', 'Source', 'Destination', 'Duration', 'Price', 'Price_per_minute']
].sort_values(by='Price_per_minute')


,Airline,Source,Destination,Duration,Price,Price_per_minute
274,Air India,Delhi,Cochin,27h 20m,4487,2.735976
10566,Air India,Delhi,Cochin,27h 20m,4487,2.735976
1571,Air India,Kolkata,Banglore,25h 55m,4647,2.988424
61,Air India,Delhi,Cochin,27h 20m,5117,3.120122
9819,Air India,Delhi,Cochin,27h 20m,5117,3.120122
...,...,...,...,...,...,...
4786,Jet Airways,Delhi,Cochin,29h 50m,13376,7.472626
8446,Jet Airways,Delhi,Cochin,29h 50m,13376,7.472626
10476,Jet Airways,Delhi,Cochin,13h 25m,6017,7.474534
9733,Jet Airways,Kolkata,Banglore,15h 45m,7064,7.475132


## 📘 Section: Anomaly Detection – Logic, Assumptions & Thresholds

## 📊 Price Anomaly Detection: Logic, Assumptions, and Thresholds

We used two transparent, statistical methods to flag pricing anomalies in the dataset: **IQR method** and **Z-score method**.

### 🔹 1. Interquartile Range (IQR) Method

**Logic:**
- Compute Q1 (25th percentile) and Q3 (75th percentile) of the price distribution.
- Calculate IQR = Q3 - Q1.
- Define anomalies as any prices:
  - Below: Q1 - 1.5 × IQR
  - Above: Q3 + 1.5 × IQR

**Assumptions:**
- Price distribution is not necessarily normal.
- Outliers exist on the high end of the price range.

**Thresholds Used:**
- Q1 = {your actual Q1 value, e.g., 7,143}
- Q3 = {your Q3 value, e.g., 11,548}
- Anomalies: `Price < lower_bound` or `Price > upper_bound`

### 🔹 2. Z-Score Method

**Logic:**
- Calculate the z-score for each price using standard deviation.
- Flag prices with z-score magnitude > 3 as anomalies.

**Assumptions:**
- Suitable for symmetric (normal-like) distributions.
- Most price values fall within ±3 standard deviations.

**Thresholds Used:**
- Anomalies: `abs(z-score) > 3`

---

## 💡 Key Insights:

- **Total price anomalies (IQR):** 94 rows
- **Most anomalies are overpriced flights.**
- No significantly underpriced flights were detected by these statistical methods.

---

## 🏷️ Scoring & Prioritization of High-Opportunity Entries

We defined a **Price Efficiency Score**:



Flights with a **low Price per Minute** are considered **potential good deals**, even if they are not statistical outliers.

We flagged:
- Top 10% of flights with the **lowest Price per Minute** as "Good Deals"
- Outliers (IQR) with high prices as potential "Overpriced Flights"

This combination allows for clear, explainable filtering of:
- **Unusually expensive entries**
- **High-value, potentially underpriced entries**

